In [10]:
from requests import exceptions
import requests
import cv2
import os
import gevent

In [11]:
searchengine = 'people mask covid'
output = 'img/mask'

In [12]:
API_KEY = "5db47142b8cd4d71bed83ea923d539b5"
MAX_RESULTS = 50
GROUP_SIZE = 250

In [13]:
URL = "https://api.bing.microsoft.com/v7.0/images/search"
       

EXCEPTIONS = {IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError, exceptions.ConnectionError,
              exceptions.Timeout}

In [14]:

term = searchengine
headers = {"Ocp-Apim-Subscription-Key": API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}

In [15]:
print("[INFO] searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()


[INFO] searching Bing API for 'people mask covid'


In [16]:
total = 0
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("[INFO] {} total results for '{}'".format(estNumResults, term))

[INFO] 50 total results for 'people mask covid'


In [17]:
def grab_page(url, ext, total):

    try:
        print("[INFO] fetching: {}".format(url))
        r = requests.get(url, timeout=30)
        p = os.path.sep.join([output, "{}{}".format(
            str(total).zfill(8), ext)])

        f = open(p, "wb")
        f.write(r.content)
        f.close()

        image = cv2.imread(p)

        if image is None:
            print("[INFO] deleting: {}".format(p))
            os.remove(p)
            return

    except Exception as e:
        if type(e) in EXCEPTIONS:
            print("[INFO] skipping: {}".format(url))
            return

In [18]:
for offset in range(0, estNumResults, GROUP_SIZE):
    print("[INFO] making request for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    print("[INFO] saving images for group {}-{} of {}...".format(
        offset, offset + GROUP_SIZE, estNumResults))
    jobs = []
    for v in results["value"]:
        total += 1
        ext = v["contentUrl"][v["contentUrl"].rfind("."):]
        url = v["contentUrl"]
        
        jobs.append(gevent.spawn(grab_page, url, ext, total))

    gevent.joinall(jobs, timeout=10)
    print(total)

[INFO] making request for group 0-250 of 50...
[INFO] saving images for group 0-250 of 50...
[INFO] fetching: https://i.insider.com/5e5e8b7cfee23d2c6d22c6d3
[INFO] skipping: https://i.insider.com/5e5e8b7cfee23d2c6d22c6d3
[INFO] fetching: https://cdn.shopify.com/s/files/1/0638/6243/products/32_0d60c626-4929-4023-a46c-b64cdd790e6b_1024x1024.jpg?v=1571438588
[INFO] skipping: https://cdn.shopify.com/s/files/1/0638/6243/products/32_0d60c626-4929-4023-a46c-b64cdd790e6b_1024x1024.jpg?v=1571438588
[INFO] fetching: https://vitalrecord.tamhsc.edu/wp-content/uploads/2020/04/cloth-face-mask-1100x625.jpg
[INFO] fetching: https://cdn.quotesgram.com/img/7/56/2038724884-2010_39_2_1.jpg
[INFO] fetching: https://media.istockphoto.com/photos/business-people-wearing-medical-masks-during-flu-or-contagious-picture-id520038579
[INFO] skipping: https://media.istockphoto.com/photos/business-people-wearing-medical-masks-during-flu-or-contagious-picture-id520038579
[INFO] fetching: http://www.masks-online.com/me